In [4]:
import os
import traceback
import pandas as pd
import geopandas as gpd
import sqlalchemy
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import LineString, MultiLineString
user='hifld_summer'
pw='hifld_summer'
name='hifld_summer'
host='moria'
port=2023
engine = sqlalchemy.create_engine(
    f'postgresql://{user}:{pw}@{host}:{port}/{name}'
    )

In [17]:
%%time
sql ="""
select build_id, geom, prim_occ from structures.alabama limit 100000
"""

try:
    
    gdf = gpd.GeoDataFrame.from_postgis(sql, con=engine,geom_col='geom') # epsg 4326

    gdf = gdf.to_crs('ESRI: 102003')

    gdf["min_rotated_rectangle"] = gdf.minimum_rotated_rectangle().boundary

    gdf = gdf.set_geometry('min_rotated_rectangle')

    gdf = gdf.drop(columns=['geom'])
    
    def explodeLine(row):
        coords = row.min_rotated_rectangle.coords 
        parts = []
        for part in zip(coords, coords[1:]):
            parts.append(LineString(part))
        return parts
            
    gdf['tempgeom'] = gdf.apply(lambda x: explodeLine(x), axis=1)
    gdf = gdf.explode('tempgeom')
    
    gdf = gpd.GeoDataFrame(data=gdf, geometry='tempgeom')
    gdf = gdf.drop('min_rotated_rectangle', axis=1)
    gdf.crs = gdf.crs

    gdf["side_length"] = gdf.length

    gdf = gdf.groupby('build_id').agg({
        'side_length': list,
        'prim_occ': 'first'  
    }).reset_index()
    gdf['rotated_long_side'] = [max(float(y) for y in x) for x in gdf['side_length']]
    gdf['rotated_short_side'] = [min(float(y) for y in x) for x in gdf['side_length']]

    gdf["rotated_ls_ratio"] = gdf["rotated_long_side"] / gdf["rotated_short_side"]
    gdf = gdf.drop(columns=['side_length'])



except Exception:
    print(traceback.print_exc())

CPU times: total: 49.5 s
Wall time: 49.7 s
